In [1]:
%%capture
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import nltk 
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
dados = pd.read_csv(data_url).replace(np.nan, '', regex=True)
documents = dados.text.count()
itens = []

### 1. Escolha uma estratégia de tokenização para a coleção que você está usando e justifique sua estratégia. É importante que você inclua decisões adicionais em relação ao que foi feito no Laboratório anterior (por exemplo, tratamento de maiúsculas/minúsculas e strings numéricas).

#### No lab anterior foi escolhida uma estratégia simples. Foram separadas as palavras consideradas úteis com uso da expressão regular `\w+`, colocando as palavras todas em caixa-baixa, sendo filtradas pelas condições de não serem *stopwords*, nem conunto de digitos numéricos, além de bigramas. Desta forma se anula boa parte do erro gerado por palavras não adequadas para análise. Contudo, ainda assim há perda de dados que seriam úteis para a análise.

#### Neste lab, será utilizado o módulo de *stemming* para português incluída na biblioteca NLTK, bem como os módulos de *stopwords* e *tokenizer*. Na tokenização desta vez consideraremos datas no formato `aaaa` e também entre maiúsculas e minúsculas, adicionando mais complexidade na análise. Primeiro faremos a tokenização mais geral, separando as palavras, depois será feita a tokenização das datas.

In [2]:
word_token = rpt(r'\w+')
year_token = rpt(r'\d{4}')

def tokenize_it(dataset, pattern, words):

    for artigo in dataset.text:
        tokens = []
        for token in pattern.tokenize(artigo):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)

#### O método `tokenize_it` faz a aplicação dos *patterns* definidos, de modo que nosso conjunto de palavras seja incrementado de acordo com o processo de tokenização. 

#### A variável `word_token` utiliza a mesma regex do outro lab. Contudo, dentro do método, não é feita nenhuma modificação no texto analizado. Desta forma, tanto palavras com maiúsculas e minúsculas são adicionadas ao vocábulo. 

### 2. Refaça a questão 2 do Laboratório anterior usando os tokens produzidos.

In [4]:
tokenize_it(dados, word_token, itens)
tokenize_it(dados, year_token, itens)

df = pd.DataFrame(itens, columns=['palavra'])
itens_df = df.palavra.value_counts().reset_index()
itens_df.columns = ['palavra', 'frequencia']
itens_df['r'] = itens_df.frequencia.rank(ascending=False, method='first')

over_1000 = len(itens_df[itens_df.frequencia > 1000])
singletons = len(itens_df[itens_df.frequencia == 1])
total_itens = len(itens)
vocab = len(set(itens))

itens_df["PR"] = round((itens_df.frequencia / total_itens) * 100, 2)
itens_df["r.PR"] = round(itens_df.r * itens_df["PR"] / 100, 3)
itens_df[:50]

,palavra,frequencia,r,PR,r.PR
0,anos,1166,1.0,0.51,0.005
1,sobre,832,2.0,0.36,0.007
2,presidente,824,3.0,0.36,0.011
3,Bolsonaro,752,4.0,0.33,0.013
4,Brasil,666,5.0,0.29,0.014
5,país,578,6.0,0.25,0.015
6,ainda,578,7.0,0.25,0.018
7,Governo,560,8.0,0.24,0.019
8,contra,476,9.0,0.21,0.019
9,pessoas,470,10.0,0.21,0.021


### 3. Aplique Stemming nos tokens produzidos e encontre 10 exemplos de falsos positivos e 10 exemplos de falsos negativos. Que impacto você acha que falsos positivos e negativos, como esses, teriam no processamento de consultas? Dê exemplos. 

### 4. Refaça a questão 3 do Laboratório anterior usando os tokens stemizados. Você percebeu alguma diferença em relação aos tokens sem stemming? Se sim, quais?